In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

import torch.optim as optim
import torchvision.models as models

import PIL.Image as Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [25]:
model = torch.load('gestures-120625-0train.pkl', map_location='cpu')
# model = torch.load('gestures.pkl')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
# 图像预处理
image_size = (128, 128)  # 图像大小设为224×224
data_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(),  # 随机水平翻转图片
    transforms.Resize(image_size),  # 重设图像大小
    # transforms.CenterCrop(image_size),  # 中间裁剪
    transforms.ToTensor(),  # 数据进转化为torch.FloadTensor，并归一化到[0, 1.0]
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [27]:
test_data = dset.ImageFolder(root="test_set_n",transform=data_transform)

In [28]:
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=40 ,shuffle=True)

In [29]:
batch = next(iter(test_loader))
images, labels = batch 
out = model(images.to(device))

In [30]:
print('labels:', labels)
print('predicts:', out.argmax(dim=1))

labels: tensor([ 1, 20, 11, 19, 19,  2, 13, 11,  3, 23, 12,  3,  2,  1, 18, 17,  4,  3,
        10, 13, 21,  2, 17,  4,  8, 15, 14,  8, 12, 20, 14, 14, 10, 20, 22, 15,
        23, 22,  8, 18])
predicts: tensor([ 1, 20, 11, 19, 19,  0, 13, 11,  3, 23, 12, 16,  2,  1, 18, 17,  4,  3,
        10, 13, 21,  2, 19,  4,  8, 19, 14, 19, 12, 20, 14, 14, 10, 20, 14,  0,
        23, 22,  8, 18])


In [31]:
# 相关函数（计算预测正确个数）
def get_num_correct(out, labels):
    return out.argmax(dim=1).eq(labels).sum().item()

correct = 0
correct += get_num_correct(out, labels.to(device))
print("val_correct:", correct/len(test_data))

val_correct: 0.825


In [ ]:
PIL_img = transforms.ToPILImage()(images[0])
PIL_img.show() # 处理后的PIL图片